In [22]:
from lxml import html
import requests
from collections import OrderedDict
import json
import argparse
import pandas as pd
import csv

def parse(input_file):
    restaurants = []
    for line in input_file:
        url = line
        response = requests.get(url).text
        parser = html.fromstring(response)
	
        XPATH_RATING = '//div[@id="ratingFilter"]//ul//li'

        XPATH_TRAVELER = '//div[@class="col segment extraWidth"]//ul//li'
        XPATH_TIME = '//div[@class="col season extraWidth"]//ul/li'
        XPATH_LANGUAGE = '//div[@class="col language extraWidth"]//ul/li'
#        XPATH_DETAILS = '//div[contains(@class,"details_tab")]//div[contains(@class, "table_section")]//text()'
        XPATH_NAME = '//h1[@property="name"]//text()'
        XPATH_HOTEL_RATING = '//span[@property="ratingValue"]//@content'
        XPATH_REVIEWS = '//a[@property="reviewCount"]/@content'
        XPATH_RANK = '//div[@class="slim_ranking"]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[contains(@class,"additional_info")]//span[contains(@class,"tabs_descriptive_text")]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[@class="additional_info"]//div'
        XPATH_LATITUDE = '//div[@class="mapContainer"]//@data-lat'
        XPATH_LONGITUDE = '//div[@class="mapContainer"]//@data-lng'
                                        
        ratings = parser.xpath(XPATH_RATING)
        travelers = parser.xpath(XPATH_TRAVELER)
        times = parser.xpath(XPATH_TIME)
        languages = parser.xpath(XPATH_LANGUAGE)
	
        raw_name = parser.xpath(XPATH_NAME)
        raw_rank = parser.xpath(XPATH_RANK)
        raw_review_count = parser.xpath(XPATH_REVIEWS)
        raw_rating = parser.xpath(XPATH_HOTEL_RATING)
#         raw_official_description = parser.xpath(XPATH_OFFICIAL_DESCRIPTION)
#        raw_details = parser.xpath(XPATH_DETAILS)
        raw_latitude = parser.xpath(XPATH_LATITUDE)
        raw_longitude = parser.xpath(XPATH_LONGITUDE)
					
        name = ''.join(raw_name).strip() if raw_name else None
        rank = ''.join(raw_rank).strip() if raw_rank else None
        review_count = ''.join(raw_review_count).strip() if raw_review_count else None
        hotel_rating = ''.join(raw_rating).strip() if raw_rating else None
#         official_description = ' '.join(' '.join(raw_official_description).split()) if raw_official_description else None
#        details = ' '.join(' '.join(raw_details).split()) if raw_details else None
        latitude = ' '.join(' '.join(raw_latitude).split()) if raw_latitude else None  
        longitude = ' '.join(' '.join(raw_longitude).split()) if raw_longitude else None 

        ratings_dict = OrderedDict()
        for rating in ratings:
            XPATH_RATING_KEY = './/div[@class="row_label"]//text()'
            XPATH_RATING_VALUE = './/span[@class="row_bar"]/following-sibling::span//text()'
            raw_rating_key = rating.xpath(XPATH_RATING_KEY)
            raw_rating_value = rating.xpath(XPATH_RATING_VALUE)
            cleaned_rating_key = ''.join(raw_rating_key).replace('\n','')
            cleaned_rating_value = ''.join(raw_rating_value).replace('\n','')
            ratings_dict.update({cleaned_rating_key:cleaned_rating_value})
    
        travelers_list = [] 
        for traveler in travelers:
            XPATH_TRAVELER_TYPE = './/label//text()'
            XPATH_TRAVELER_COUNT = './/span//text()'
            raw_traveler_type = traveler.xpath(XPATH_TRAVELER_TYPE)
            raw_traveler_count = traveler.xpath(XPATH_TRAVELER_COUNT)
            cleaned_traveler_type = ''.join(raw_traveler_type).replace('\n','').split(' ')
            cleaned_traveler_type = cleaned_traveler_type[0]
            cleaned_traveler_count = ''.join(raw_traveler_count).replace('\n','')
            travelers_list.append((cleaned_traveler_type,cleaned_traveler_count))
        
        times_list = []
        for time in times:
            XPATH_TIME_TYPE = './/label//text()'
            XPATH_TIME_COUNT = './/span//text()'
            raw_time_type = time.xpath(XPATH_TIME_TYPE)
            raw_time_count = time.xpath(XPATH_TIME_COUNT)
            cleaned_time_type = ''.join(raw_time_type).replace('\n','').split(' ')
            cleaned_time_type = cleaned_time_type[0]
            cleaned_time_count = ''.join(raw_time_count).replace('\n','')
            times_list.append((cleaned_time_type, cleaned_time_count))
        
        language_list = []
        for language in languages:
            XPATH_LANGUAGE_TYPE = './/label//text()'
            XPATH_LANGUAGE_COUNT = './/span//text()'
            raw_language_type = language.xpath(XPATH_LANGUAGE_TYPE)
            raw_language_count = language.xpath(XPATH_LANGUAGE_COUNT)
            cleaned_language_type = ''.join(raw_language_type).replace('\n','').split(' ')
            cleaned_language_type = cleaned_language_type[0]
            cleaned_language_count = ''.join(raw_language_count).replace('\n','')
            language_list.append(cleaned_language_type)
        language_list = language_list[1:]


        data = {'name':name,
				'rank':rank,
				'rating':hotel_rating,
                'review_count':review_count,
                'latitude':latitude,
                'longitude':longitude,
#                 'official_description':official_description,
                  'travelers':travelers_list,
                  'times of year':times_list,
                  'languages':language_list
#                  'details':details
	    }
        restaurants.append(data)
        
        
#    columns = ['property_title', 'rank', 'rating', 'review_count', 'location', 'official_description', 'travelers', 'times', 'langauges']
    df = pd.DataFrame(restaurants)
    return df



input_file = open('links.csv', 'r')
links = []
for i in input_file:
    links.append('https://www.tripadvisor.com'+i)
scraped_data = parse(links)
scraped_data

,languages,latitude,longitude,name,rank,rating,review_count,times of year,travelers
0,"[English(230), Spanish(20), French(16), ]",31.952438,35.937958,Pizza Roma Cafe,#1 of 544 Restaurants in Amman,4.5,302,"[(Mar-May, (64)), (Jun-Aug, (39)), (Sep-Nov, (...","[(Families, (24)), (Couples, (56)), (Solo, (56..."
1,"[English(260), Arabic(4), French(2), ]",31.963032,35.89816,Brisket,#2 of 544 Restaurants in Amman,4.5,269,"[(Mar-May, (55)), (Jun-Aug, (58)), (Sep-Nov, (...","[(Families, (56)), (Couples, (24)), (Solo, (18..."
2,"[English(536), Arabic(40), Chinese, ]",31.951878,35.9332,Habibah Sweets,#3 of 544 Restaurants in Amman,4.5,671,"[(Mar-May, (156)), (Jun-Aug, (119)), (Sep-Nov,...","[(Families, (149)), (Couples, (74)), (Solo, (4..."
3,"[English(676), Italian(31), Arabic(28), ]",31.952278,35.92017,Fakhreldin Restaurant,#4 of 544 Restaurants in Amman,4.5,789,"[(Mar-May, (202)), (Jun-Aug, (146)), (Sep-Nov,...","[(Families, (165)), (Couples, (79)), (Solo, (1..."
4,"[English(605), Italian(33), Chinese, ]",31.94924,35.929058,Sufra Restaurant,#5 of 544 Restaurants in Amman,4.5,704,"[(Mar-May, (180)), (Jun-Aug, (134)), (Sep-Nov,...","[(Families, (126)), (Couples, (87)), (Solo, (2..."
5,"[English(460), Italian(79), Spanish(56), ]",31.992243,35.866783,Tawaheen al-Hawa,#6 of 544 Restaurants in Amman,4.5,713,"[(Mar-May, (133)), (Jun-Aug, (89)), (Sep-Nov, ...","[(Families, (111)), (Couples, (77)), (Solo, (9..."
6,"[English(272), French(4), Italian(3), ]",31.952156,35.91342,Chestnut Restaurant & Pub,#8 of 544 Restaurants in Amman,4.5,283,"[(Mar-May, (79)), (Jun-Aug, (57)), (Sep-Nov, (...","[(Families, (21)), (Couples, (26)), (Solo, (9)..."
7,"[English(56), Japanese(1)]",31.961784,35.8701,V Lounge & Restaurant,#7 of 544 Restaurants in Amman,5,57,"[(Mar-May, (52)), (Jun-Aug, (2)), (Sep-Nov, (1...","[(Families, (9)), (Couples, (7)), (Solo, (3)),..."
8,"[English(1,038), Italian(109), French(46), ]",31.952234,35.93188,Hashem,#9 of 544 Restaurants in Amman,4.5,1365,"[(Mar-May, (290)), (Jun-Aug, (226)), (Sep-Nov,...","[(Families, (168)), (Couples, (215)), (Solo, (..."
9,"[English(79), French(2), Italian(2), ]",31.953451,35.884483,Melograno,#10 of 544 Restaurants in Amman,5,84,"[(Mar-May, (27)), (Jun-Aug, (7)), (Sep-Nov, (2...","[(Families, (20)), (Couples, (14)), (Solo, (2)..."


In [23]:
with open('restaurants_details3.csv', 'w') as csvfile:
    scraped_data.to_csv(csvfile,columns = ['latitude','longitude','name'], encoding='utf-8')

In [24]:
scraped_data.dropna()

,languages,latitude,longitude,name,rank,rating,review_count,times of year,travelers
0,"[English(230), Spanish(20), French(16), ]",31.952438,35.937958,Pizza Roma Cafe,#1 of 544 Restaurants in Amman,4.5,302,"[(Mar-May, (64)), (Jun-Aug, (39)), (Sep-Nov, (...","[(Families, (24)), (Couples, (56)), (Solo, (56..."
1,"[English(260), Arabic(4), French(2), ]",31.963032,35.89816,Brisket,#2 of 544 Restaurants in Amman,4.5,269,"[(Mar-May, (55)), (Jun-Aug, (58)), (Sep-Nov, (...","[(Families, (56)), (Couples, (24)), (Solo, (18..."
2,"[English(536), Arabic(40), Chinese, ]",31.951878,35.9332,Habibah Sweets,#3 of 544 Restaurants in Amman,4.5,671,"[(Mar-May, (156)), (Jun-Aug, (119)), (Sep-Nov,...","[(Families, (149)), (Couples, (74)), (Solo, (4..."
3,"[English(676), Italian(31), Arabic(28), ]",31.952278,35.92017,Fakhreldin Restaurant,#4 of 544 Restaurants in Amman,4.5,789,"[(Mar-May, (202)), (Jun-Aug, (146)), (Sep-Nov,...","[(Families, (165)), (Couples, (79)), (Solo, (1..."
4,"[English(605), Italian(33), Chinese, ]",31.94924,35.929058,Sufra Restaurant,#5 of 544 Restaurants in Amman,4.5,704,"[(Mar-May, (180)), (Jun-Aug, (134)), (Sep-Nov,...","[(Families, (126)), (Couples, (87)), (Solo, (2..."
5,"[English(460), Italian(79), Spanish(56), ]",31.992243,35.866783,Tawaheen al-Hawa,#6 of 544 Restaurants in Amman,4.5,713,"[(Mar-May, (133)), (Jun-Aug, (89)), (Sep-Nov, ...","[(Families, (111)), (Couples, (77)), (Solo, (9..."
6,"[English(272), French(4), Italian(3), ]",31.952156,35.91342,Chestnut Restaurant & Pub,#8 of 544 Restaurants in Amman,4.5,283,"[(Mar-May, (79)), (Jun-Aug, (57)), (Sep-Nov, (...","[(Families, (21)), (Couples, (26)), (Solo, (9)..."
7,"[English(56), Japanese(1)]",31.961784,35.8701,V Lounge & Restaurant,#7 of 544 Restaurants in Amman,5,57,"[(Mar-May, (52)), (Jun-Aug, (2)), (Sep-Nov, (1...","[(Families, (9)), (Couples, (7)), (Solo, (3)),..."
8,"[English(1,038), Italian(109), French(46), ]",31.952234,35.93188,Hashem,#9 of 544 Restaurants in Amman,4.5,1365,"[(Mar-May, (290)), (Jun-Aug, (226)), (Sep-Nov,...","[(Families, (168)), (Couples, (215)), (Solo, (..."
9,"[English(79), French(2), Italian(2), ]",31.953451,35.884483,Melograno,#10 of 544 Restaurants in Amman,5,84,"[(Mar-May, (27)), (Jun-Aug, (7)), (Sep-Nov, (2...","[(Families, (20)), (Couples, (14)), (Solo, (2)..."


In [25]:
scraped_data2 = pd.DataFrame(scraped_data.dropna())

In [26]:
lat = [(float(x)) for x in scraped_data2['latitude']]
long = [(float(x)) for x in scraped_data2['longitude']]

In [27]:
import gmplot
# __, __ center map , __ zoom of map
gmap = gmplot.GoogleMapPlotter(31.950000, 35.920000, 14)
gmap.scatter(lat, long,'#3B0B39', size=40, marker=False)
#gmap.heatmap(lat,long)
gmap.draw("Amman_map.html")

In [28]:
#https://pypi.python.org/pypi/gmplot/1.0.5

In [29]:
#https://github.com/tcassou/gmaps    map examples

In [30]:
from collections import Counter
import re
#def convert(string):
#    return Counter(re.findall('[A-Z][a-z]*', string))
import ast
languages_dictsList = []
for i in scraped_data2['languages']:
    lang = (re.findall('[A-Z][a-z]*', str(i)))
    freq = (re.findall('[1-9][0-9]*', str(i)))
    dictionary = dict(zip(lang, freq))
    languages_dictsList.append(dictionary)
lang_df = pd.DataFrame(languages_dictsList)
new_names = [(i,'lang_'+i) for i in lang_df.iloc[:, 0:].columns.values]
lang_df.rename(columns = dict(new_names), inplace=True) #adding prefix to know what type of cat
resulting_df = scraped_data2.join(lang_df)
resulting_df.fillna(value=0, inplace=True) #final dataframe with languages of reviews as categories

In [31]:
travelers_dictsList = []
for i in scraped_data2['travelers']:
    trav = (re.findall('[A-Z][a-z]*', str(i)))
    freq = (re.findall('[1-9][0-9]*', str(i)))
    dictionary = dict(zip(trav, freq))
    travelers_dictsList.append(dictionary)
travelers_df = pd.DataFrame(travelers_dictsList)
new_names = [(i,'travType_'+i) for i in travelers_df.iloc[:, 0:].columns.values] 
travelers_df.rename(columns = dict(new_names), inplace=True) #adding prefix to know what type of cat
resulting_df2 = resulting_df.join(travelers_df)
resulting_df2.fillna(value=0, inplace=True) #types of travelers added as categories 

In [32]:
import sklearn
import sklearn.cluster
import numpy as np
matrix = resulting_df2[['latitude','longitude']].as_matrix()
np.nan_to_num(matrix)
km = sklearn.cluster.KMeans(n_clusters=4)
km.fit(matrix)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
results = pd.DataFrame([resulting_df2.index,labels]).T

In [33]:
print(results)

       0  1
0      0  0
1      1  2
2      2  0
3      3  0
4      4  0
5      5  1
6      6  2
7      7  1
8      8  0
9      9  2
10    10  1
11    11  0
12    12  0
13    13  0
14    14  2
15    15  2
16    16  3
17    17  2
18    18  2
19    19  0
20    20  0
21    21  0
22    22  1
23    23  1
24    24  2
25    25  2
26    26  2
27    27  2
28    28  2
29    29  0
..   ... ..
540  540  0
541  541  2
542  542  0
543  543  0
544  544  0
545  545  1
546  546  2
547  547  1
548  548  0
549  549  2
550  550  1
551  551  0
552  552  0
553  553  0
554  554  2
555  555  2
556  556  3
557  557  2
558  558  2
559  559  0
560  560  0
561  561  0
562  562  1
563  563  1
564  564  2
565  565  2
566  566  2
567  567  2
568  568  2
569  569  0

[570 rows x 2 columns]


In [34]:
print(labels)

[0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2
 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0
 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0
 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2
 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0
 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0
 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2
 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2
 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0
 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2
 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3
 2 2 0 0 0 1 1 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1
 2 2 2 2 2 0 0 2 0 0 0 1 2 1 0 2 1 0 0 0 2 2 3 2 2 0 0 0 1 1 2 2 2 2 2 0 0
 2 0 0 0 1 2 1 0 2 1 0 0 